# Computing chemical properties

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 4th, 2022</p>

```{attention}
Always make sure you are using the [latest version of Reaktoro](https://anaconda.org/conda-forge/reaktoro). Otherwise, some new features documented on this website will not work on your machine and you may receive unintuitive errors. Follow these [update instructions](updating_reaktoro_via_conda) to get the latest version of Reaktoro!
```

You have now learned how to create a {{ChemicalSystem}} object and {{ChemicalState}} objects representing the chemical state of that system. We will now demonstrate how thermodynamic and chemical properties of the system, its phases, and constituting species can be calculated using class {{ChemicalProps}}.

Let's start with a simple chemical system containing a single aqueous phase:

In [1]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C"))

system = ChemicalSystem(db, solution)

Let's create a chemical state for this system representing a 1 molal NaCl solution with 0.7 molal dissolved CO{{_2}}:

In [2]:
state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 0.7, "mol")

equilibrate(state)

print(state)

+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |    333.1500 |    K |
| Pressure        |     15.1988 |  bar |
| Charge:         | -1.3146e-16 |  mol |
| Element Amount: |             |      |
| :: H            |  1.1101e+02 |  mol |
| :: C            |  7.0000e-01 |  mol |
| :: O            |  5.6906e+01 |  mol |
| :: Na           |  1.0000e+00 |  mol |
| :: Cl           |  1.0000e+00 |  mol |
| Species Amount: |             |      |
| :: CO3-2        |  4.9431e-11 |  mol |
| :: H+           |  6.9406e-04 |  mol |
| :: H2O          |  5.5506e+01 |  mol |
| :: CO2          |  6.5830e-01 |  mol |
| :: (CO2)2       |  2.0503e-02 |  mol |
| :: HCO3-        |  4.7002e-04 |  mol |
| :: CH4          |  1.0000e-16 |  mol |
| :: Cl-          |  1.0000e+00 |  mol |
| :: H2           |  1.0000e-16 |  mol |
| :: Na+          |  9.9978e-01 |  mol |
| :: NaCO3-       |  4.4528e-09 |  mol |
| :: NaHCO3     

```{note}
We used the convenience method {{equilibrate}} above to quickly bring the {{ChemicalState}} object `state` into a state of chemical equilibrium. We will later learn how to use Reaktoro's Gibbs energy minimization algorithm to perform a wide variety of chemical equilibrium calculations using class {{EquilibriumSolver}} together with class {{EquilibriumSpecs}}, which allows us to enforce different equilibrium constraints at equilibrium (e.g., fixed pH, pE, Eh, gas fugacity, enthalpy, internal energy, volume, etc.).
```

We have seen that class {{ChemicalState}} is not responsible for computing thermodynamic and chemical properties of the system. It's primary use is for preparing initial states for chemical equilibrium/kinetics calculations and also to store the result of these computations.

Computing chemical properties of {{ChemicalSystem}} objects is a task for class {{ChemicalProps}}:

In [3]:
props = ChemicalProps(state)
print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     333.1500 |         K |
| Pressure                               |      15.1988 |       bar |
| Volume                                 |   1.0595e-03 |        m3 |
| Gibbs Energy                           |  -73110.8025 |         J |
| Enthalpy                               |   -1197.0604 |         J |
| Entropy                                |     215.8599 |       J/K |
| Internal Energy                        |   -2807.3552 |         J |
| Helmholtz Energy                       |  -74721.0973 |         J |
| Charge                                 |  -1.3146e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

By default, when defining a chemical system using {{AqueousPhase}}, {{GaseousPhase}}, {{LiquidPhase}}, {{SolidPhase}}, {{MineralPhase}}, {{CondensedPhase}}, {{IonExchangePhase}}, etc., these objects are created with **ideal activity models**. That's why you should see unit activity coefficients above for all species. 

We will learn later how to specify more sophisticated and accurate activity models for each phase, but out of curiosity, we will define Pitzer's activity model from {cite:t}`Harvie1980,Harvie1984`. Let's also consider a gaseous phase and some mineral phases. For the gaseous phase, we shall use Peng-Robinson's equation of state {cite}`Peng1976` as its activity model. 

We will need to reconstruct the {{ChemicalSystem}} object, as well as the {{ChemicalState}} and {{ChemicalProps}} objects:

In [4]:
solution = AqueousPhase(speciate("H O Na Cl C Ca Si"))
solution.set(ActivityModelPitzer())

gases = GaseousPhase()
gases.set(ActivityModelPengRobinsonPhreeqcOriginal())

minerals = MineralPhases()

system = ChemicalSystem(db, solution, gases, minerals)

state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O"    , 1.0, "kg")
state.set("Na+"    , 1.0, "mol")
state.set("Cl-"    , 1.0, "mol")
state.set("CO2"    , 0.7, "mol")
state.set("Calcite", 1.0, "g")
state.set("Quartz" , 1.0, "g")

equilibrate(state)

props = ChemicalProps(state)

print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     333.1500 |         K |
| Pressure                               |      15.1988 |       bar |
| Volume                                 |   1.8888e-03 |        m3 |
| Gibbs Energy                           |  -79530.4785 |         J |
| Enthalpy                               |    4934.7046 |         J |
| Entropy                                |     253.5350 |       J/K |
| Internal Energy                        |    2064.0204 |         J |
| Helmholtz Energy                       |  -82401.1626 |         J |
| Charge                                 |  -2.5853e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

The results displayed above should now be a little more interesting.

Now consider a scenario where your {{ChemicalState}} object `state` has changed, for example 1 mol of NaCl has been added to it and the temperature has increased by 10 °C. The chemical properties of the system must then be updated as shown below:

In [5]:
state.temperature(70.0, "celsius")
state.add("Na+", 1.0, "mol")
state.add("Cl-", 1.0, "mol")

props.update(state)

print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     343.1500 |         K |
| Pressure                               |      15.1988 |       bar |
| Volume                                 |   1.9428e-03 |        m3 |
| Gibbs Energy                           |  -82114.8348 |         J |
| Enthalpy                               |    8335.6371 |         J |
| Entropy                                |     263.5887 |       J/K |
| Internal Energy                        |    5382.7573 |         J |
| Helmholtz Energy                       |  -85067.7146 |         J |
| Charge                                 |  -3.6955e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

The printed table above shows many different properties. Let's now use some methods from class {{ChemicalProps}} to retrieve some specific properties for the system, its phases, and its species:

In [6]:
print("SYSTEM'S TEMPERATURE     :", props.temperature()                         , "K"     )
print("SYSTEM'S PRESSURE        :", props.pressure()                            , "Pa"    )
print("SYSTEM'S VOLUME          :", props.volume()                              , "m3"    )
print("AQUEOUS PHASE'S DENSITY  :", props.phaseProps("AqueousPhase").density()  , "kg/m3" )
print("AQUEOUS PHASE'S MASS     :", props.phaseProps("AqueousPhase").mass()     , "kg"    )
print("AQUEOUS PHASE'S VOLUME   :", props.phaseProps("AqueousPhase").volume()   , "m3"    )
print("AQUEOUS PHASE'S ENTHALPY :", props.phaseProps("AqueousPhase").enthalpy() , "J/mol" )
print("GASEOUS PHASE'S DENSITY  :", props.phaseProps("GaseousPhase").density()  , "kg/m3" )
print("GASEOUS PHASE'S MASS     :", props.phaseProps("GaseousPhase").mass()     , "kg"    )
print("GASEOUS PHASE'S VOLUME   :", props.phaseProps("GaseousPhase").volume()   , "m3"    )
print("GASEOUS PHASE'S ENTHALPY :", props.phaseProps("GaseousPhase").enthalpy() , "J/mol" )
print("ACTIVITY H+              :", props.speciesActivity("H+")                           )
print("ACTIVITY H+ (LOG10)      :", props.speciesActivityLg("H+")                         )
print("ACTIVITY COEFF. CO2      :", props.speciesActivityCoefficient("CO2")               )
print("ACTIVITY COEFF. HCO3-    :", props.speciesActivityCoefficient("HCO3-")             )
print("FUGACITY COEFF. CO2(g)   :", props.speciesActivityCoefficient("CO2(g)")            )
print("FUGACITY COEFF. H2O(g)   :", props.speciesActivityCoefficient("H2O(g)")            )
print("FUGACITY COEFF. O2(g)    :", props.speciesActivityCoefficient("O2(g)")             )

SYSTEM'S TEMPERATURE     : 343.15 K
SYSTEM'S PRESSURE        : 1.51988e+06 Pa
SYSTEM'S VOLUME          : 0.00194284 m3
AQUEOUS PHASE'S DENSITY  : 1058.59 kg/m3
AQUEOUS PHASE'S MASS     : 1.12711 kg
AQUEOUS PHASE'S VOLUME   : 0.00106473 m3
AQUEOUS PHASE'S ENTHALPY : 717.96 J/mol
GASEOUS PHASE'S DENSITY  : 24.6018 kg/m3
GASEOUS PHASE'S MASS     : 0.0215941 kg
GASEOUS PHASE'S VOLUME   : 0.000877743 m3
GASEOUS PHASE'S ENTHALPY : 8028.09 J/mol
ACTIVITY H+              : 1.5507e-05
ACTIVITY H+ (LOG10)      : -4.80947
ACTIVITY COEFF. CO2      : 1.35237
ACTIVITY COEFF. HCO3-    : 0.452266
FUGACITY COEFF. CO2(g)   : 0.947632
FUGACITY COEFF. H2O(g)   : 0.903559
FUGACITY COEFF. O2(g)    : 1.01054


```{tip}
Access the link {{ChemicalProps}} to find out all methods available in class `ChemicalProps` to get the exact data you need!
```